In [1]:
from langchain_community.document_loaders import PyPDFDirectoryLoader

loader = PyPDFDirectoryLoader("./data_2")
docs = loader.load()
docs

[Document(page_content='Course Title\nPython for Data Science and Machine Learning Bootcamp\nThe Data Science Course: Complete Data Science Bootcamp 2024 \nR Programming A-Z™: R For Data Science With Real Exercises! \nStatistics for Data Science and Business Analysis \nData Science A-Z: Hands-On Exercises & ChatGPT Prize [2024] \nMachine Learning, Data Science and Generative AI with Python \nPython A-Z™: Python For Data Science With Real Exercises!\nComplete A.I. & Machine Learning, Data Science Bootcamp \nData Science and Machine Learning Bootcamp with R \nIntroduction to Machine Learning for Data Science \nBuild Responsive Real-World Websites with HTML and CSS \nHTML and CSS for Beginners - Build a Website & Launch ONLINE \nResponsive Web Design: HTML5 + CSS3 for Entrepreneurs 2018 \nWeb Design for Beginners: Real World Coding in HTML & CSS \nHTML5 and CSS3 Fundamentals \nLearn HTML5 Programming From Scratch \nBuild Websites from Scratch with HTML & CSS \nIntroduction to Cloud Comput

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1200, chunk_overlap = 250)
documents = text_splitter.split_documents(docs)

In [3]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

In [4]:
db = FAISS.from_documents(documents, OllamaEmbeddings())

In [5]:
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser



In [6]:
from dotenv import load_dotenv
import os

load_dotenv()
groq_api_key=os.getenv('GROQ_API_KEY')


In [7]:
llm = ChatGroq(groq_api_key = groq_api_key,
               model_name="Llama3-8b-8192")

In [41]:
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
"""
Act as helpfull Talent Transformation assistant and suggest the best Courses to take based on the given 
context, given {skills} and given {experience}. I want all the output in JSON format.


Prepare 3 sets of Mandatory courses by total learning hours:
- Set 1 (1-day plan): Course duration < 3 hours
- Set 2 (3-day plan): Course duration between 3 and 20 hours
- Set 3 (5-day plan): Course duration > 20 hours



For each suggested course, provide the following:
- Course Name
- Course Link
- Course Length
- Learner Level

Please ensure consistency in providing course details and consider the inclusion of mandatory courses in the recommendations.

Suggest the courses only based on the context knowledge
Keep your course recomandation ground in the facts of the context.
If the context does not contain the suitable courses according to the given {skills} then return "No course Found"

Be as concise as possible when giving answers. Do not anounce that you will answering. 
Do not say "Let's get started", or "Based on the skills you provided, I would recommend the 
following courses to help you enhance your knowledge and skills : " or "Here are some courses"
and do not introduce your answer. Just write the your answer.

<context>
{context}
<context>

Skills: {skills}

Experience: {experience}

"""
)

In [42]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [43]:
retriever = db.as_retriever()

parser = JsonOutputParser()
setup = RunnableParallel(context=retriever, skills = RunnablePassthrough(), experience = RunnablePassthrough())

chain = setup | prompt | llm 

In [47]:
response = chain.invoke({"skills": "python and AI",
              "experience": 2.5})

print(response)

InternalServerError: Error code: 503 - {'error': {'message': 'Service Unavailable', 'type': 'internal_server_error'}}

In [ ]:
import json
data_dict = json.loads(response)

# Convert the Python dictionary to JSON format
json_data = json.dumps(data_dict, indent=2)

print(json_data)